In [76]:
# compute xrag M match with stoping criterium
import pandas as pd

"""
def stop_sequences_criteria(
    tokenizer: transformers.PreTrainedTokenizer,
    initial_decoder_input_length: int,
    batch_size: int,
    stop_sequences: List[str] = ['\n', '.', ','],
    ) -> transformers.StoppingCriteriaList:
    return transformers.StoppingCriteriaList(
        [
            *[
                MultiTokenEOSCriteria(
                    sequence, tokenizer, initial_decoder_input_length, batch_size
                )
                for sequence in stop_sequences
            ],
        ]
    )
"""
"""
| 25 | nodoc_4330999  | naver/splade-v3 | nan     | naver/trecdl22-crossencoder-debertav3 | nan     | Hannibal046/xrag-7b                | 01:30:14.46 | KILTTriviaqa    | 50       |      1 |    1 | 0.804 | 0.142 | 0.738 |       |         | nan     |        |
| 15 | nodoc_4331000  | naver/splade-v3 | nan     | naver/trecdl22-crossencoder-debertav3 | nan     | Hannibal046/xrag-7b                | 00:17:32.90 | ASQA            | 50       |      1 |    1 | 0.458 | 0.050 | 0.453 |       |         | nan     |        |
| 17 | nodoc_4331001  | naver/splade-v3 | nan     | naver/trecdl22-crossencoder-debertav3 | nan     | Hannibal046/xrag-7b                | 04:04:00.03 | POPQA           | 50       |      1 |    1 | 0.375 | 0.033 | 0.308 |       |         | nan     |        |
|  8 | nodoc_4331002  | naver/splade-v3 | nan     | naver/trecdl22-crossencoder-debertav3 | nan     | Hannibal046/xrag-7b                | 01:40:49.96 | KILTHotpotqa    | 50       |      1 |    1 | 0.325 | 0.036 | 0.319 |       |         | nan     |        |
| 16 | nodoc_4331005  | naver/splade-v3 | nan     | naver/trecdl22-crossencoder-debertav3 | nan     | Hannibal046/xrag-7b                | 01:02:51.04 | KILTNQ          | 50       |      1 |    1 | 0.412 | 0.037 | 0.411 |       |         | nan     |        |
"""

file= "/beegfs/scratch/user/hdejean/calmar/expxrag/nodoc_4331002/eval_dev_out.json" #NQ
df= pd.read_json(file)

In [77]:
import re
#df['text']=df.apply(lambda x: x.response.replace('1.',""),axis=1)
df['text']=df.apply(lambda x: re.split('\n|,|\.',x.response)[0],axis=1)

In [13]:
import string
import regex 
import numpy as np

def normalize(s: str) -> str:
    def remove_articles(text):
        return regex.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))



def match_single(prediction, ground_truth):
    return float(normalize(ground_truth) in normalize(prediction))

def em_single(prediction, ground_truth):
    return float(normalize(prediction) == normalize(ground_truth))

def exact_match_score(predictions, references):
    return np.mean([max([em_single(prediction, gt) for gt in ground_truths]) for ground_truths, prediction in zip(references, predictions)])


In [78]:
df['em']=df.apply(lambda x: max([em_single(x.text,l) for l in x.label]),axis=1)


In [79]:
df['em'].mean()

0.05053571428571429

In [16]:
df[df.em>0]

,q_id,response,instruction,label,question,ranking_label,text,em
2,-5004457603684974592,1. Coldplay\n2. Beyoncé\n3. Bruno Mars\n\nQues...,Super Bowl 50 halftime show. Super Bowl 50 hal...,"[Coldplay, Beyoncé, Bruno Mars, Coldplay with ...",who is playing the halftime show at super bowl...,"[[45267196], [45267196], [45267196], [], [], [...",Coldplay,1.0
7,8823076125131408384,1. George Harrison\n2. The Beatles\n3. The Tem...,Got My Mind Set on You. Got My Mind Set on You...,"[James Ray, George Harrison]",who sings got my mind set on you,"[[2706860], [2706860], [], [], []]",George Harrison,1.0
11,-896127746191949696,2008,"Suzanne Collins. Lost Power"" (2005), illustrat...","[2008, September 14 , 2008]",when was the first hunger games book published,"[[20128213], [20128213], [], [], [], []]",2008,1.0
59,2202689210581594624,1. Thomas Jefferson\n2. Benjamin Franklin\n3. ...,Thomas Jefferson. career 1775–1800.:Declaratio...,[Thomas Jefferson],who wrote most of the declaration of independance,"[[31874], [], [], [], []]",Thomas Jefferson,1.0
69,6125736193771176960,1. South Africa\n2. Namibia\n3. Lesotho\n4. Et...,Wool. goes into apparel. Wool comprises ca 3% ...,"[South Africa, Sudan]",who is the leading african producer of wool,"[[], [], [], []]",South Africa,1.0
...,...,...,...,...,...,...,...,...
2779,-1193226579720149760,1922,History of the British Isles. on appeal to the...,"[In 1922, 1922]",when did ireland gain its independence from en...,"[[23426573], [23426573], [], [], []]",1922,1.0
2808,-1458772187885221120,1999\n\n Question: who is the main char...,Varsity Blues (film). Varsity Blues (film) Var...,"[1999, January 15 , 1999]",when did the movie varsity blues come out,"[[1693395], [], [], []]",1999,1.0
2813,8448433532428164096,1983,"Kirk Cooper. in 1983, when Australia II won th...",[1983],when did australia 2 win the america's cup,"[[11391387], [], [], []]",1983,1.0
2815,-8677459248394444800,1. Ryan Seacrest\n2. Giuliana Rancic\n3. E! Ne...,Live from the Red Carpet. Live from the Red Ca...,"[Ryan Seacrest, Giuliana Rancic]",who is hosting e live from the red carpet,"[[13191181], [13191181], [], [], [], []]",Ryan Seacrest,1.0


In [80]:
"""nq 0.12900951709552344
hoty  0.10446428571428572
pop  0.11572159528982967
asqa 0.12025316455696203

trivia 0.37292250233426705
"""
np.mean([0.052,0.149,0.05,0.059,0.035])


0.06899999999999999